
# DevX -  AI Augmented Software Development Platform  (Ashish K Jain)


## Multi Agent Event Driven Software Development platform using LlamaIndex and OpenAI’s GPT-4o & O3-Mini"

### DevX is transforming the way engineering teams build, debug, and optimize software. This notebook demonstrates an AI-powered development workflow that leverages LlamaIndex to coordinate two advanced models: GPT-4o-mini for analyzing user input, and O3-Mini for tasks such as design, coding, DevOps, and more.

### Key Features - Purpose: 
    Multi-agent, event-driven architecture.
    AI-assisted generation of user stories, designs, code, DevOps configurations, and test cases.
    Feedback-driven refinement at every stage of the software development lifecycle.

### While AI-assisted coding is becoming increasingly common, this notebook introduces a unique event-driven, agentic workflow. Engineers can initiate the development process with a simple requirement, and DEVX intelligently guides them through each phase of the software development lifecycle—including user stories, system design, code, DevOps integration, and test case generation.

### At every stage, engineers can provide feedback to refine and adjust the output according to their needs. There's no need for manual prompt engineering—developers can simply offer feedback to regenerate content, proceed to the next step, or stop the workflow at any point.

## DevX WorkFlow
<img src="../images/DevX-Flow.png" alt="DevX WorkFlow" width="1000" height="1000">

#### This cell lists the required Python libraries for the notebook. It includes commands to install OpenAI, Gradio, and various LlamaIndex-related packages. These libraries are essential for interacting with OpenAI models, building workflows, and creating a user interface.

In [ ]:
#pip install openai
#pip install gradio
#pip install llama-parse
#pip install llama-index-utils-workflow
#pip install llama-index-core 

#### This cell imports the necessary Python libraries and modules. It includes OpenAI for API interaction, LlamaIndex for workflow management, and utility functions for visualizing workflows.

In [ ]:
import openai
import os
from openai import OpenAI
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Event,
    Context
)
from llama_index.utils.workflow import draw_all_possible_flows
from llama_index.core.workflow import InputRequiredEvent, HumanResponseEvent
import gradio as gr
import asyncio
import threading
import queue
import time

#### We must load the OpenAI key from the environment variable and set it into api_key. We will get this key while registering with OPENAI. We then instantiate the Open AI client using the key. We will also define two variables for two models.

In [ ]:
openai.api_key  = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=openai.api_key)

#### This cell applies nest_asyncio to allow nested asynchronous loops, which is necessary for running asynchronous workflows in a Jupyter Notebook.

In [ ]:
import nest_asyncio
nest_asyncio.apply()

#### This function interacts with the OpenAI model to generate responses based on a given prompt and model type. It simplifies the process of querying the model.

In [ ]:
def call_model(model, prompt, model_type):
    model_response = model.chat.completions.create(model=model_type,
                                                     messages=[{"role":"user","content": prompt}])
    response_content = model_response.choices[0].message.content
    return response_content

####  This section defines custom event classes for different stages of the workflow, such as ProductManagerEvent, SoftwareArchitectEvent, and DevOpsEngineerEvent. These classes encapsulate the data required for each stage.

In [ ]:
class ProductManagerEvent(Event):
    requirement: str

class UserStoryFeedbackEvent(Event):
    feedback: str
        
class SoftwareArchitectEvent(Event):
    userStories: str

class DesignFeedbackEvent(Event):
    feedback: str

class SoftwareEngineerEvent(Event):
    design: str

class CodeFeedbackEvent(Event):
    feedback: str 

class DevOpsEngineerEvent(Event):
    reviewedCode: str

class DevOpsFeedbackEvent(Event):
    feedback: str 


class TestEngineerEvent(Event):
    devopscode: str

class TestCaseFeedbackEvent(Event):
    feedback: str 
        
class ProgressEvent(Event):
    msg: str

#### This section implements the AIDevXWorkflow class, which defines the steps of the AI-driven development workflow. Each step is annotated with @step and performs a specific task, such as generating user stories, accepting feedback, or progressing to the next stage.

#### Key Steps:
    set_up: Initializes the workflow and validates user requirements.
    generateUserStories: Generates user stories based on requirements.
    get_userStories_feedback: Processes user feedback on user stories.
    generateDesign: Generates a high-level system design.
    get_design_feedback: Processes user feedback on the design.
    generateCode: Generates code based on the design.
    get_code_feedback: Processes user feedback on the code.
    generateDevOps: Generates DevOps configurations.
    get_devOps_code_feedback: Processes user feedback on DevOps configurations.
    generateTestCases: Generates test cases for the system.
    get_testcase_feedback: Processes user feedback on test cases.

#### Right now, each step in the DevX flow is powered by direct OpenAI calls through LlamaIndex. But you can think of them as early-stage agents—each designed with a specific function and responsibility. With RAG and persistent memory, these will evolve into intelligent AI agents that reason across stages, recall past decisions, and collaborate like real teammates.”*


In [ ]:
# The AIDevXWorkflow class is a workflow automation framework designed to guide the software development 
# lifecycle using AI-driven steps. It leverages OpenAI models to generate user stories, designs, code, 
# DevOps configurations, and test cases while incorporating user feedback at each stage.

class AIDevXWorkflow(Workflow):
    
    GPT_4O_MINI_MODEL = 'gpt-4o-mini'
    O3_MINI_MODEL = 'o3-mini'
    model: OpenAI
    
    # declare a function as a step
    @step
    async def set_up(self, ctx: Context, ev: StartEvent) -> ProductManagerEvent:
        
        # Check for if developers requirement
        if not ev.query:
            raise ValueError("User Requirements are missing")
        
        self.model = OpenAI(api_key=openai.api_key)
        await ctx.set("requirement", str(ev.query))
        pevent = ProgressEvent(
            msg="Workflow started. Generating User-Stories",
            event_type="GENERATE_USER_STORIES"
        )
    
        ctx.write_event_to_stream(pevent)
        
        return ProductManagerEvent(requirement=ev.query)
    
    @step
    async def generateUserStories(self, ctx: Context, ev: ProductManagerEvent | UserStoryFeedbackEvent) -> InputRequiredEvent:
        requirement = await ctx.get("requirement")
        user_feedback_on_users_stories = ""
        if hasattr(ev,"feedback"):
            user_feedback_on_users_stories = ev.feedback
        
        product_manager_agent = f"""You are an experienced Software Product Manager responsible for converting 
            high-level business requirements into detailed user stories. Your output will be used by the Software Architect 
            to propose a high-level design and technology stack.

            Instructions:

            1-) Analyze the given high-level requirements and break them down into well-defined user stories using the INVEST criteria (Independent, Negotiable, Valuable, Estimable, Small, Testable).
            Each user story should follow the standard format:
            2-) "As a [user role], I want [functionality] so that [benefit]."
            3-) Include Acceptance Criteria for each user story using the Gherkin format (Given-When-Then).
            4-) Provide any assumptions or constraints that should be considered.
            5-) Please also mention the user story number on each story so we can refer that with these numbers.

            BELOW ARE THE REQUIREMENTS
            {requirement}

            You might have generated user stories before and user might have given some feedback for the same. If USER INSTRUCTIONS is
            empty assume its first time if not then follow the feedback for regeneration.

            USER INSTRUCTIONS
            {user_feedback_on_users_stories}

            """
        userStories = call_model(self.model,product_manager_agent,self.O3_MINI_MODEL)
        
        
        # Save the user stories for later
        await ctx.set("userStories", str(userStories))
        
        pevent = ProgressEvent(
            msg="User Stories generated successfully",
            event_type="userstory_feedback"
        )
        ctx.write_event_to_stream(pevent)
        # Let's get a human in the loop
        return InputRequiredEvent(
            prefix="\n\n"+"Above are the user-stories. Do you have any feedback please let us know. You can also choosed 1 , 2 or more storeis for generate code",
            result=userStories,
            event_type="userstory_feedback"
        )

    
    
    @step
    async def get_userStories_feedback(self, ctx: Context, ev: HumanResponseEvent) -> UserStoryFeedbackEvent | SoftwareArchitectEvent | StopEvent:
        if (ev.event_type == "userstory_feedback"):
            response = ev.response
            feedback = f"""
                You have received some human feedback on the user stories. User either can give some feedback and ask
                for the regeneration user stories again or can pick one or more stories for design generation.
                <feedback>
                {response}
                </feedback>
                If there's any feedback on user stories and ask for regenration, respond with just the word 'FEEDBACK'.
                If ask for go for design generation for one or more stories say 'OKAY
                Any If ask for finish or any other thing say 'CLOSE'.

            """
            result = call_model(self.model,feedback,self.GPT_4O_MINI_MODEL)

            verdict = result.strip() 
            
            if (verdict == "OKAY"):
                pevent = ProgressEvent(
                    msg=f"User has provided update on the user stories OK. Generating Design",
                    event_type="GENERATE_DESIGN"
                )
                ctx.write_event_to_stream(pevent)
                await ctx.set("user_instruction_for_design_generation", str(response))
                return SoftwareArchitectEvent(userStories = await ctx.get("userStories"))
            if (verdict == "CLOSE"):
                pevent = ProgressEvent(
                    msg=f"User want to exit the workflow. Stopping the Workflow. Please start new workflow by submitting new requirement",
                    event_type="STOP"
                )
                ctx.write_event_to_stream(pevent)
                return StopEvent(result = "Excellent you have got what you need. Best of luck for your product")
            else:
                pevent = ProgressEvent(
                    msg=f"User has provided feedback on the user stories. So Generating User story again with feedback",
                    event_type="GENERATE_USER_STORIES"
                )
                ctx.write_event_to_stream(pevent)
                return UserStoryFeedbackEvent(feedback=ev.response)
    
    
    @step
    async def generateDesign(self, ctx: Context, ev: SoftwareArchitectEvent | DesignFeedbackEvent) -> InputRequiredEvent:
        requirement = await ctx.get("requirement")
        userStories = await ctx.get("userStories")
        user_instruction_for_design_generation = await ctx.get("user_instruction_for_design_generation")
        user_feedback_on_design = ""
        if hasattr(ev,"feedback"):
            user_feedback_on_design = ev.feedback
    
        ai_architect_prompt = f"""You are an experienced Software Architect, responsible for designing a scalable, high-performance 
            based on the provided user stories. Your output will serve as input for the AI Engineer agent, which will 
            implement the system based on your design.

            Your task - 

            1-) Define the high-level architecture, specifying key components and their responsibilities.
            2-) Recommend the technology stack, considering factors like scalability, performance, and maintainability.
            3-) Design the data flow, if any
            4-) Specify databases and caching mechanisms (e.g., PostgreSQL, Redis) and justify their use. If any
            5-) Outline inter-service communication (e.g., REST APIs).
            6-) Include security considerations and best practices for reliability and fault tolerance.
            7-) You also need to check if their is need to built any new service or component or existing component can help 
            the design.

            You will given all the user-storeis for system and user instruction which one he want to design, it might include one, 
            two or all user stories. You will also additionaly given initial requirements on which these user stories has been 
            drived which might help understand the full picture.

            REQUIREMENTS
            {requirement}

            ALL USER STORIES
            {userStories}

            USER INSTRUCTIONS_FOR_DESIGN_GENERATION
            {user_instruction_for_design_generation}
            
            You might have generated design before and user might have given some feedback for the same. If USER INSTRUCTIONS is
            empty assume its first time if not then follow the feedback for regeneration.

            USER INSTRUCTIONS_ON_DESIGN_REGERATION
            {user_feedback_on_design}

            """
        
        design = call_model(self.model,ai_architect_prompt,self.O3_MINI_MODEL)
        # Save the user stories for later
        await ctx.set("design", str(design))
        
        pevent = ProgressEvent(
            msg="Design generated successfully",
            event_type="design_feedback"
        )
        ctx.write_event_to_stream(pevent)
        #Let's get a human in the loop
        return InputRequiredEvent(
            prefix="\n\n"+"Above are the design for selected user-story . Do you have any feedback please let us know or You can go for code generation",
            result=design,
            event_type="design_feedback"
        )
    
    
    
    @step
    async def get_design_feedback(self, ctx: Context, ev: HumanResponseEvent) -> DesignFeedbackEvent | SoftwareEngineerEvent | StopEvent:
        if (ev.event_type == "design_feedback"):
            response = ev.response
            feedback = f"""
                You have received some human feedback on the design. User either can give some feedback and ask
                for the regeneration design again or ask for the next step that is code generation for design.
                <feedback>
                {response}
                </feedback>
                If there's any feedback on design and ask for regenration, respond with just the word 'FEEDBACK'.
                If ask for go for code generation or ask for next step say 'OKAY'.
                Any If ask for finish or any other thing say 'CLOSE'.

            """
            result = call_model(self.model,feedback,self.GPT_4O_MINI_MODEL)

            verdict = result.strip()
            
            if (verdict == "OKAY"):
                pevent = ProgressEvent(
                    msg=f"User has provided update on the Design OK. Generating Code",
                    event_type="GENERATE_CODE"
                )
                ctx.write_event_to_stream(pevent)
                await ctx.set("user_instruction_for_code_generation", str(response))
                return SoftwareEngineerEvent(design= await ctx.get("design"))
            if (verdict == "CLOSE"):
                pevent = ProgressEvent(
                    msg=f"User want to exit the workflow. Stopping the Workflow. Please start new workflow by submitting new requirement",
                    event_type="STOP"
                )
                ctx.write_event_to_stream(pevent)
                return StopEvent(result = "Excellent you have got what you need. Best of luck for your product")
            else:
                pevent = ProgressEvent(
                    msg=f"User has provided feedback on the Design. So Generating Design again with feedback",
                    event_type="GENERATE_DESIGN"
                )
                ctx.write_event_to_stream(pevent)
                return DesignFeedbackEvent(feedback=ev.response)
    
    
    
    @step
    async def generateCode(self, ctx: Context, ev: SoftwareEngineerEvent | CodeFeedbackEvent) -> InputRequiredEvent:
        userStories = await ctx.get("userStories")
        design = await ctx.get("design")
        user_instruction_for_code_generation = ctx.get("user_instruction_for_code_generation")
        user_feedback_on_code = ""
        if hasattr(ev,"feedback"):
            user_feedback_on_code = ev.feedback
        
        ai_engineer_prompt = f"""
            You are an experienced AI engineer responsible for implementing a software design into a working codebase. 
            Given a AI ARCHITETCT SYSTEM DESIGN RESPONSE, your task is to:

            1-) Generate clean, modular, and well-documented code following the given architecture.
            2-) Provide step-by-step setup instructions to help developers run the system.
            3-) Ensure security, scalability, and maintainability in the implementation.

            Instructions for Code Implementation - 

            Read the provided software design carefully, understanding its architecture, 
            components, data flow, and technology stack. You also need to intelligently find out what needs to be build 
            and any recommendation for AI Engineer.

            Generate the necessary code for each module, ensuring:

            1-) Clear separation of concerns (e.g., API layer, business logic, data access).
            2-) Proper validation, error handling, and logging.
            3-) Adherence to security best practices (e.g., authentication, encryption, rate limiting).
            4-) Scalability and performance optimizations.
            5-) Include comments and documentation for maintainability.
            6-) Please dont generate any code for infra or DevOps as we will have next step for the same.

            Generate configuration files, if required (e.g., environment variables, database schemas, API keys).

            Ensure inter-service communication is correctly implemented (e.g., REST APIs, message queues, database queries).

            You will also additionaly given all user stories which might help understand the full picture and design for user 
            selected user stories. You will also have instruction for code generation if any.

            ALL USER STORIES
            {userStories}

            AI ARCHITETCT SYSTEM DESIGN RESPONSE ON 1, 2 or MORE USER STORIES
            {design}
            
            USER INSTRUCTIONS_FOR_CODE_GENERATION
            {user_instruction_for_code_generation}
            
            You might have generated code before and user might have given some feedback for the same. If USER INSTRUCTIONS is
            empty assume its first time if not then follow the feedback for regeneration.

            USER INSTRUCTIONS_ON_CODE_REGERATION
            {user_feedback_on_code}
            
            """
        
    
        code = call_model(self.model,ai_engineer_prompt,self.O3_MINI_MODEL)
        await ctx.set("code", str(code))
        pevent = ProgressEvent(
            msg="Code generated successfully",
            event_type="code_feedback"
        )
        ctx.write_event_to_stream(pevent)
        return InputRequiredEvent(
            prefix="\n\n"+"Above are the code for selected design . Do you have any feedback please let us know or You can go for DevOps code generation process",
            result=code,
            event_type="code_feedback"
        )
    
    
    @step
    async def get_code_feedback(self, ctx: Context, ev: HumanResponseEvent) -> CodeFeedbackEvent | DevOpsEngineerEvent | StopEvent:
        if (ev.event_type == "code_feedback"):
            response = ev.response
            feedback = f"""
                You have received some human feedback on the code. User either can give some feedback and ask
                for the regeneration code again or ask for the next step that is DevOps code for design.
                <feedback>
                {response}
                </feedback>
                If there's any feedback on code and ask for regenration, respond with just the word 'FEEDBACK'.
                If ask for go for DevOps code generation or ask for next step say 'OKAY'.
                Any If ask for finish or any other thing say 'CLOSE'.

            """
            result = call_model(self.model,feedback,self.GPT_4O_MINI_MODEL)

            verdict = result.strip()
            if (verdict == "OKAY"):
                pevent = ProgressEvent(
                    msg=f"User has provided update on the Code OK. Generating DevOps Code",
                    event_type="GENERATE_DEVOPS_CODE"
                )
                ctx.write_event_to_stream(pevent)
                await ctx.set("user_instruction_for_devops_code_generation", str(response))
                return DevOpsEngineerEvent(reviewedCode= await ctx.get("code"))
            if (verdict == "CLOSE"):
                pevent = ProgressEvent(
                    msg=f"User want to exit the workflow. Stopping the Workflow. Please start new workflow by submitting new requirement",
                    event_type="STOP"
                )
                ctx.write_event_to_stream(pevent)
                return StopEvent(result = "Excellent you have got what you need. Best of luck for your product")
            else:
                pevent = ProgressEvent(
                    msg=f"User has provided feedback on the Code. So Generating Code again with feedback",
                    event_type="GENERATE_CODE"
                )
                ctx.write_event_to_stream(pevent)
                return CodeFeedbackEvent(feedback=ev.response)
    
    


    
    @step
    async def generateDevOps(self, ctx: Context, ev: DevOpsEngineerEvent | DevOpsFeedbackEvent) -> InputRequiredEvent:
        design = await ctx.get("design")
        code = await ctx.get("code")
        user_instruction_for_devops_code_generation = ctx.get("user_instruction_for_devops_code_generation")
        user_feedback_on_devops_code = ""
        if hasattr(ev,"feedback"):
            user_feedback_on_devops_code = ev.feedback
        
        ai_devOps_prompt = f"""

            You are an expert AI DevOps Engineer responsible for automating CI/CD workflows, deployment pipelines, and 
            infrastructure provisioning. Your tasks include:

            1-) CI/CD Pipeline Configuration: Set up GitHub Actions, Jenkins, or other CI/CD tools for automated testing and deployment.
            2-) Deployment & Monitoring: Ensure smooth rollouts with proper rollback mechanisms, environment-specific configurations, and monitoring setup.
            3-) Infrastructure Automation: Auto-generate Dockerfiles, Kubernetes manifests, Terraform scripts, and Helm charts for cloud deployment.
            4-) Security & Compliance: Implement secrets management, RBAC policies, and best practices for secure deployments.
            5-) Observability & Alerting: Configure logging, monitoring, and alerting using tools like Prometheus, Grafana, and ELK stack.


            You will given the system design for user story and generated code for the same. You will also 
            given additional user instruction from user which infrastruture, devops tooling, deployemnt policies, 
            IAC will be used.

            DESIGN_FOR_A_USER_STORY
            {design}
            
            CODE_FOR_THE_SYSTEM_DESIGN
            {code}

            USER INSTRUCTIONS_FOR_DEVOPS_CODE_GENERATION
            {user_instruction_for_devops_code_generation}
            
            You might have generated devops code before and user might have given some feedback for the same. If USER INSTRUCTIONS is
            empty assume its first time if not then follow the feedback for regeneration.

            USER INSTRUCTIONS_ON_DEVOPS_CODE_REGERATION
            {user_feedback_on_devops_code}


        """
        devops_code = call_model(self.model,ai_devOps_prompt,self.O3_MINI_MODEL)
        
        await ctx.set("devops_code", str(devops_code))
        pevent = ProgressEvent(
            msg="DevOps Code generated successfully",
            event_type="devops_code_feedback"
        )
        ctx.write_event_to_stream(pevent)
        return InputRequiredEvent(
            prefix="\n\n"+"Above are the DevOps code for selected design . Do you have any feedback please let us know or You can go for test cases generation",
            result=devops_code,
            event_type="devops_code_feedback"
        )
        

    
    
    @step
    async def get_devOps_code_feedback(self, ctx: Context, ev: HumanResponseEvent) -> DevOpsFeedbackEvent | TestEngineerEvent | StopEvent:
        if (ev.event_type == "devops_code_feedback"):
            response = ev.response
            feedback = f"""
                You have received some human feedback on the devops code. User either can give some feedback and ask
                for the regeneration devops code again or ask for the next step that is test case generation process.
                <feedback>
                {response}
                </feedback>
                If there's any feedback on devops code and ask for regenration, respond with just the word 'FEEDBACK'.
                If ask for go for Test case generation or ask for next step say 'OKAY'.
                Any If ask for finish or any other thing say 'CLOSE'.

            """
            result = call_model(self.model,feedback,self.GPT_4O_MINI_MODEL)

            verdict = result.strip()

            print(f"LLM says the devops code feedback verdict was {verdict}")
            if (verdict == "OKAY"):
                pevent = ProgressEvent(
                    msg=f"User has provided update on the DEVOPS Code OK. Generating Test Cases",
                    event_type="GENERATE_TESTCASE"
                )
                ctx.write_event_to_stream(pevent)
                await ctx.set("user_instruction_for_testcase_generation", str(response))
                return TestEngineerEvent(devopscode= await ctx.get("devops_code"))
            if (verdict == "CLOSE"):
                pevent = ProgressEvent(
                    msg=f"User want to exit the workflow. Stopping the Workflow. Please start new workflow by submitting new requirement",
                    event_type="STOP"
                )
                ctx.write_event_to_stream(pevent)
                return StopEvent(result = "Excellent you have got what you need. Best of luck for your product")
            else:
                pevent = ProgressEvent(
                    msg=f"User has provided feedback on the DevOps Code. So Generating DevOps Code again with feedback",
                    event_type="GENERATE_DEVOPS_CODE"
                )
                ctx.write_event_to_stream(pevent)
                return DevOpsFeedbackEvent(feedback=ev.response)
    
    @step
    async def generateTestCases(self, ctx: Context, ev: TestEngineerEvent | TestCaseFeedbackEvent) -> InputRequiredEvent:
        design = await ctx.get("design")
        code = await ctx.get("code")
        devops_configuration = await ctx.get("devops_code")
        
        user_instruction_for_testcase_generation = ctx.get("user_instruction_for_testcase_generation")
        user_feedback_on_testcase_generation = ""
        if hasattr(ev,"feedback"):
            user_feedback_on_testcase_generation = ev.feedback
            
        ai_expert_tester = f"""
            As an AI Expert Tester, your goal is to ensure software reliability, performance, and security by 
            conducting automated testing across the entire development lifecycle. You will receive design specifications for a user story,
            application code for system design, and DevOps configurations as input. 

            Your Responsibilities:
            1-) Analyze the Design, Verify functional and non-functional requirements, Identify potential edge cases, 
            missing scenarios, and security loopholes.

            2-) Review the Code - Generate comprehensive unit tests ensuring high code coverage. Detect logical inconsistencies, error handling issues, and potential regressions.
            Validate security best practices (e.g., OWASP Top 10 vulnerabilities).

            3-) Test the DevOps Code: Review CI/CD pipelines for robustness and security. Generate load testing scripts to 
            validate system scalability.

            4-) Ensure Dockerfiles, Kubernetes manifests, and Terraform scripts are optimized for deployment if any

            5-) Automate the Testing Process: Use appropriate frameworks (Jest, PyTest, JUnit, k6, JMeter, Locust).

            6-) Simulate real-world load, stress, and performance conditions.

            7-) Integrate tests into the CI/CD pipeline for continuous validation.

            8-) Provide Insights & Reports: Generate detailed reports on test coverage, execution results, and detected issues.

            9-) Suggest improvements for code quality, performance, and deployment stability.
            
            You will also given additional user instruction from user for test case generation if any.
        

            DESIGN_FOR_A_USER_STORY
            {design}
            
            CODE_FOR_THE_SYSTEM_DESIGN
            {code}
            
            DEVOPS_CONFIGURATION
            {devops_configuration}
            
            USER INSTRUCTIONS_FOR_TESTCASE_GENERATION
            {user_instruction_for_testcase_generation}
            
            You might have generated test cases before and user might have given some feedback for the same. If USER INSTRUCTIONS is
            empty assume its first time if not then follow the feedback for regeneration.

            USER INSTRUCTIONS_ON_TESTCASE_REGERATION
            {user_feedback_on_testcase_generation}

            """
        
        testcases = call_model(self.model,ai_expert_tester,self.O3_MINI_MODEL)
        
        
        await ctx.set("testcases", str(testcases))
        pevent = ProgressEvent(
            msg="TestCases generated successfully",
            event_type="testcase_feedback"
        )
        ctx.write_event_to_stream(pevent)
        return InputRequiredEvent(
            prefix="\n\n"+"Above are the Test cases for selected design and code. Do you have any feedback please let us know or You can exit the workflow",
            result=testcases,
            event_type="testcase_feedback"
        )
    
    
    @step
    async def get_testcase_feedback(self, ctx: Context, ev: HumanResponseEvent) -> TestCaseFeedbackEvent | StopEvent:
        if (ev.event_type == "testcase_feedback"):
            response = ev.response
            feedback = f"""
                You have received some human feedback on the test case generation. User either can give some feedback and ask
                for the regeneration test cases again or ask for the finish the workflow
                <feedback>
                {response}
                </feedback>
                If there's any feedback on test cases generation and ask for regenration, respond with just the word 'FEEDBACK'.
                If ask for finish and any other thing say 'CLOSE'.

            """
            result = call_model(self.model,feedback,self.GPT_4O_MINI_MODEL)

            verdict = result.strip()
            if (verdict == "CLOSE"):
                pevent = ProgressEvent(
                    msg=f"User has provided fine with the DEVOPS Code OK. Stopping the Workflow",
                    event_type="STOP"
                )
                ctx.write_event_to_stream(pevent)
                return StopEvent(result = "Excellent you have got what you need. Best of luck for your product")
            else:
                pevent = ProgressEvent(
                    msg=f"User has provided feedback on the Test Cases. So Generating TestCases again with feedback",
                    event_type="GENERATE_TESTCASE"
                )
                ctx.write_event_to_stream(pevent)
                return TestCaseFeedbackEvent(feedback=ev.response)
        

### Workflow Execution  Defines functions to execute the workflow in a background thread and handle user interactions. Key Functions:
    run_workflow_thread: Runs the workflow asynchronously in a separate thread.
    get_label_for_event: Maps event types to user-friendly labels with icons.
    Gradio Callbacks:
        start_workflow: Starts the workflow when the user clicks "Start Workflow."
        refresh_status: Refreshes the workflow status.
        submit_input: Handles user input during the workflow.

#### This cell defines global variables to hold streaming updates, such as progress messages, prompts, and user input.

In [ ]:
# Globals to hold streaming updates.
global_progress = ""    # This will accumulate progress messages
global_prompt   = ""    # This will hold the current prompt info
global_current_event = ""
input_queue = queue.Queue()   # For sending user responses from the UI to the workflow

#### This cell defines a function to run the workflow in a background thread using an asyncio event loop.

In [ ]:
# This function runs workflow “event loop” in a background thread.
# It uses a new asyncio event loop to await events.
def run_workflow_thread(requirement):
    
    
    # Create and set an event loop BEFORE calling the workflow.
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    
    # instantiate your workflow (make sure you have imported your workflow code)
    workflow = AIDevXWorkflow(timeout=None, verbose=False)

    async def process_events():
        global global_progress, global_prompt, global_current_event
        try:
            handler = workflow.run(query=requirement)
            async for ev in handler.stream_events():
                if isinstance(ev, ProgressEvent):
                    # Append the message to the progress log
                    global_progress += ev.msg + "\n"
                    global_current_event = ev.event_type
                elif isinstance(ev, InputRequiredEvent):
                    # Set the prompt info (this may include a design story, code snippet, etc.)
                    global_prompt = ev.result + "\n" + ev.prefix
                    global_current_event = ev.event_type
                    # Wait in a blocking fashion for user input from the queue.
                    # (Since we are in a separate thread, this blocking call is acceptable.)
                    response = None
                    while response is None:
                        try:
                            # Use a short timeout so we can check repeatedly (alternatively, you could block)
                            response = input_queue.get(timeout=0.1)
                        except queue.Empty:
                            await asyncio.sleep(0.1)
                    # Echo user submission into the progress log if desired
                    global_progress += "User responded  : " + response + "\n"
                    # Clear the prompt once an answer is received
                    global_prompt = ""
                    # Send the response event back into your workflow
                    handler.ctx.send_event(
                        HumanResponseEvent(
                            response=response,
                            event_type=ev.event_type
                        )
                    )
        except asyncio.CancelledError:
            raise
        finally:
            # Ensure any pending tasks are properly cleaned up
            for task in asyncio.all_tasks(loop):
                if task is not asyncio.current_task(loop):
                    task.cancel()
            
            # Wait for all tasks to complete
            pending = asyncio.all_tasks(loop)
            if pending:
                await asyncio.gather(*pending, return_exceptions=True)
    

    # Run the event loop with proper task cleanup
    try:
        task = loop.create_task(process_events())
        loop.run_until_complete(task)
    except Exception as e:
        print(f"Error occurred: {e}")
    finally:
        try:
            # Cancel any pending tasks
            pending = asyncio.all_tasks(loop)
            for task in pending:
                task.cancel()
            # Allow cancelled tasks to complete
            if pending:
                loop.run_until_complete(
                    asyncio.gather(*pending, return_exceptions=True)
                )
        finally:
            loop.close()

#### This cell defines a function to map event types to user-friendly labels with icons.

In [ ]:
def get_label_for_event(event_type):
    """Map event types to user-friendly labels with icons"""
    event_labels = {
        "userstory_feedback": "📝 Generated User Stories",
        "GENERATE_USER_STORIES": "📝 User Stories Generation....",
        "design_feedback": "🎨 Generated Design Specification",
        "GENERATE_DESIGN": "🎨 Design Specification Generation....",
        "code_feedback": "💻 Generated Code",
        "GENERATE_CODE": "💻 Code Generation....",
        "testcase_feedback": "🧪 Generated Test Cases",
        "GENERATE_TESTCASE": "🧪 Test Cases Generation....",
        "devops_code_feedback": "📋 Generated DevOps Code",
        "GENERATE_DEVOPS_CODE": "📋 DevOps Code Generation....",
        "STOP": "⌛ Waiting for New Flow to Start",
    }
    return event_labels.get(event_type, "📊 Workflow Progress & Output")

#### This cell defines a callback function to start the workflow when the user clicks "Start Workflow."


In [ ]:
# Gradio callbacks:
# a. start_workflow: called when user clicks “Start Workflow”
def start_workflow(requirement):
    global global_progress, global_prompt
    # Reset globals whenever a new run is started
    global_progress = ""
    global_prompt = ""
    # Start your workflow in a background thread:
    t = threading.Thread(target=run_workflow_thread, args=(requirement,))
    t.start()
    return "Workflow started…"

#### This cell defines a callback function to refresh the status of the workflow.

In [ ]:

# b. refresh_status: called (by a Refresh button or a client-side timer) to update UI components.
def refresh_status():
    # Return the current progress log and prompt text.
    requirement = ""  # Initialize requirement variable
    if(global_current_event == "STOP"):
        requirement: ""  
    else:
        requirement = gr.update()  # Keep requirement as is
    return {
        status_output: global_progress,
        prompt_output: gr.update(
            value=global_prompt,
            label=get_label_for_event(global_current_event)
        ),
        requirement_input:requirement
        
    }

#### This cell defines a callback function to handle user input when the "Submit" button is clicked.

In [ ]:
# c. submit_input: when the user enters their response and clicks “Submit”
def submit_input(user_input):
    # Put the user input in the queue for the workflow thread.
    if user_input and user_input.strip() != "":
        input_queue.put(user_input)
    return ""

####  User Interface (Gradio Integration).   Builds a user-friendly interface using Gradio for interacting with the workflow. Key Components:
    Input fields for user requirements and feedback.
    Buttons for starting the workflow, refreshing status, and submitting input.
    Output areas for displaying progress and results.
    Custom CSS:
        Enhances the UI with a modern, clean design.

In [ ]:
# Define custom CSS for a modern DEVX UI with a light background
custom_css = """
    body {
        background-color: #f5f5f5;
        color: #212121;
        font-family: 'Inter', sans-serif;
    }
    .title {
        color: #2e7d32;
        text-align: center;
        font-size: 26px;
        font-weight: bold;
        margin-bottom: 15px;
    }
    .info-box {
        background-color: #e8f5e9;
        padding: 15px;
        border-radius: 10px;
        box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);
        margin-bottom: 15px;
    }
    .button-primary {
        background-color: #2e7d32 !important;
        color: #ffffff !important;
        font-weight: bold !important;
        border-radius: 6px !important;
    }
    .output-box {
        font-family: 'Monaco', 'Courier New', monospace;
        background-color: #ffffff;
        color: #212121;
        padding: 15px;
        border-radius: 8px;
        min-height: 400px;
        white-space: pre-wrap;
        border: 1px solid #2e7d32;
        overflow-y: auto;
    }
    .center-content {
        display: flex;
        justify-content: center;
        align-items: center;
        margin-bottom: 20px;
    }
"""

# Build the enhanced Gradio UI
with gr.Blocks(theme=gr.themes.Soft(), css=custom_css) as demo:
    

    gr.Markdown("<div class='title'>🚀 DevX AI-Augmented Software Development</div>")
    gr.Markdown(
        "<div class='info-box'>"
        "<b>Enter a requirement and start the AI-driven development workflow "
        "You'll see updates (e.g., design, user stories, code, test cases) in the progress panel. "
        "When required, provide input in the right panel and continue the workflow.</b> "
        "</div>"
    )

    with gr.Row():
        with gr.Column(scale=1):
            
            requirement_input = gr.Textbox(label="📌 Requirement", 
                                           placeholder="Describe your requirement here...",
                                           lines=5)
            
            start_button = gr.Button("🚀 Start Workflow", elem_classes="button-primary")
            
            status_output = gr.Textbox(label="📢 Streaming of Events", 
                                       lines=15,
                                       interactive=False, 
                                       elem_classes="output-box")
            refresh_button = gr.Button("🔄 Refresh Status",elem_classes="button-primary")  # Moved below the progress log
            
        with gr.Column(scale=1):
            
            prompt_output = gr.Textbox(label="📝 DEVX Output", 
                                       lines=30, 
                                       interactive=False, 
                                       show_label=True,
                                       elem_classes="output-box")
          
            user_input = gr.Textbox(label="✍ Your Input", 
                                    placeholder="Type your response here...", 
                                    lines=2)
            submit_button = gr.Button("✅ Submit", elem_classes="button-primary")
          

    # Wire up the buttons:
    start_button.click(start_workflow, inputs=requirement_input, outputs=status_output)
    refresh_button.click(refresh_status, outputs=[status_output, prompt_output, requirement_input])
    submit_button.click(submit_input, inputs=user_input, outputs=user_input)


#### This cell launches the Gradio application, which provides a user interface for interacting with the AI-driven development workflow.



In [ ]:
# Launch the Gradio app
demo.launch(
    share=False, 
    server_port=9000, 
    prevent_thread_lock=True
)

#### This cell closes the Gradio app if it is running.

In [ ]:
import gradio as gr
demo.close()

#### This section uses the draw_all_possible_flows function to visualize the workflow. It helps in understanding the flow of events and dependencies between different stages.

In [ ]:
draw_all_possible_flows(
    AIDevXWorkflow, 
    filename="../workflows/workflow.html"
)

####  Helper function (extract_html_content  )to format and display code or HTML outputs in the Jupyter Notebook. These functions enhance the readability of the generated outputs. It renders the workflow diagram generated in the previous step, providing a graphical representation of the workflow.

In [ ]:
from IPython.display import display, HTML

def extract_html_content(filename):
    try:
        with open(filename, 'r') as file:
            html_content = file.read()
            html_content = f""" <div style="width: 100%; height: 800px; overflow: hidden;"> {html_content} </div>"""
            return html_content
    except Exception as e:
        raise Exception(f"Error reading file: {str(e)}")

html_content = extract_html_content("../workflows/workflow.html")
display(HTML(html_content), metadata=dict(isolated=True))